In [0]:
import pickle
import os
import pandas as pd
import multiprocessing as proc
from skimage.data import imread
from sklearn.ensemble import RandomForestClassifier

In [0]:
model = pickle.load(open('/home/maansi/input/camera_model.pkl', 'rb'))
test_images_folder = '/home/maansi/input/check'
test_images = []
for fname in sorted(os.listdir(test_images_folder)):
    test_images.append(fname)
infer = pd.DataFrame(test_images, columns=['fname'])

In [0]:
def feature_extract(m, lock):
    while True:
        images_list = m.get()
        if images_list is None:
            break
        if type(images_list) is tuple:
            img = imread('/home/maansi/input/train'/images_list[0]/images_list[1])
            key = images_list[1]
        else:
            img = imread('/home/maansi/input/check/'+images_list)
            key = images_list

        if img.shape == (2,):
            img = img[0]

        features[key] = (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                           img[:, :, 0].std(),  img[:, :, 1].std(),  img[:, :, 2].std())    

In [0]:
feature_cols = ['channel1_mean', 'channel2_mean', 'channel3_mean', 'channel1_std', 'channel2_std', 'channel3_std']

for col in feature_cols:
    infer[col] = None

In [0]:
parallel_cores = 8

features = proc.Manager().dict()

m = proc.Queue(maxsize=parallel_cores)
lock = proc.Lock()
process_pool = proc.Pool(parallel_cores, initializer=feature_extract, initargs=(m, lock))

for i in test_images:
    m.put(i)
    
for _ in range(parallel_cores):  
    m.put(None)
process_pool.close()
process_pool.join()

features = dict(features)

In [0]:
for n, col in enumerate(feature_cols):
    infer[col] = infer['fname'].apply(lambda x: features[x][n])

In [0]:
X_test = infer[feature_cols].values

In [0]:
y_pred = model.predict(X_test)
print(test_images)
print(y_pred)

['(iP4s)26.jpg']
['iPhone-4s']
